# **Convolutional Neural Networks (CNN)**
ใน Notebook นี้เราจะมาลงมือทดลองสร้างและเทรน CNN เบื้องต้นด้วยไลบรารี่ Pytorch, Pytorch Lightning, และ FastAI รวมถึงการทำ Transfer Learning กัน

In [ ]:
import torch

torch.cuda.is_available() # ตรวจสอบว่ามี CUDA หรือ GPU ที่สามารถใช้ได้หรือไม่
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolutional-neural-network

import torch.nn as nn
import torch.nn.functional as F


class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            # dimension of color channels (3 = RGB)
            in_channels = 1, 
            out_channels = 6,
            # size of square kernels (use (w,h) for non-square windows)
            kernel_size = 5, 
            stride = 2, # step size
            )
        self.pool = nn.MaxPool2d(
            kernel_size = 2, # size of square kernel (same as before)
            stride = 2,
            )
        self.conv2 = nn.Conv2d(
            in_channels = 6, # equals to out_channels of the previous conv layer 
            out_channels = 16,
            kernel_size = 5,
            ) # use the default stride = 1

        ## Linear layers after flatten the convolutional filters
        self.fc1 = nn.Linear(
            # in_features 
            # = (previous out_channels) X (kernel_width) X (kernel_height)
            in_features = 16 * 5 * 5,
            out_features = 120,
            )
        self.fc2 = nn.Linear(
            120, # in_features as a positional argument (no need to specify)
            84,
            )
        # 10 is the number of classes (0-9)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = ConvNet()
net

# **Transfer Learning**
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#convnet-as-fixed-feature-extractor


### **Transfer Learning ด้วย Pytorch**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
class TransferNet(nn.Module):
    def __init__(self, backbone, num_classes):
        super(TransferNet, self).__init__()
        self.model = backbone
        # "freeze" weights ของ backbone ไม่ให้เปลี่ยนแปลง
        for param in self.model.parameters():
            param.requires_grad = False
        # เปลี่ยน layer สุดท้ายของโมเดลที่เลือกใช้งาน
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.model(x)

# โหลดโมเดล ResNet18 จาก torchvision.models และส่งเข้า class TransferNet ของเรา
backbone = torchvision.models.resnet18(pretrained=True)
net = TransferNet(backbone = backbone, num_classes = 2)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
# กำหนด loop การเทรนโมเดล
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Iterate over the dataloader
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Report loss every 100 batch
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


from torch.optim.lr_scheduler import ReduceLROnPlateau
epochs = 50
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss() # loss function for classification
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate) # Optimizer
scheduler = ReduceLROnPlateau(optimizer, "min")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer)
    val_loss = test_loop(val_dataloader, net, loss_fn, optimizer)
    scheduler.step(val_loss)
print("Done!")

### **Transfer Learning ด้วย Pytorch Lightning**

In [ ]:
# !pip install pytorch_lightning

In [ ]:
import pytorch_lightning as pl
import torch.nn.functional as F

class TransferNet(pl.LightningModule): # inherit จาก pl.LightningModule แทน nn.Module
    def __init__(self, backbone, num_classes, learning_rate):
        super(TransferNet, self).__init__()
        self.model = backbone
        self.learning_rate = learning_rate
        # "freeze" weights ของ backbone ไม่ให้เปลี่ยนแปลง
        for param in self.model.parameters():
            param.requires_grad = False
        # เปลี่ยน layer สุดท้ายของโมเดลที่เลือกใช้งาน
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

    # กำหนด loop การเทรนโมเดลภายใน class
    def training_step(self, batch, batch_idx):
        x, y = batch
        prediction = self(x)
        loss = F.cross_entropy(prediction, y)
        return loss

    # กำหนดเลือก optimizer ที่นี่เช่นเดียวกัน
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.learning_rate)

In [ ]:
trainer = pl.Trainer(max_epochs=50)
backbone = torchvision.models.resnet18(pretrained=True)
net = TransferNet(
    backbone = backbone,
    num_classes = 2,
    learning_rate=1e-3) 

trainer.fit(net, train_dataloaders=train_dataloader)

### **Transfer Learning ด้วย FastAI**

In [ ]:
# https://www.analyticsvidhya.com/blog/2021/05/training-state-of-the-art-deep-learning-models-with-fast-ai/

from fastai import *
from fastai.vision import *

# กำหนด folder ที่เก็บข้อมูลไว้ให้กับ ImageDataLoaders
# ภายใต้ path ที่กำหนดจะมี 2 folders ย่อย คือ training/ และ validation/ 
dls = ImageDataLoaders.from_folder(path=path, 
                                    train='training',
                                    valid='validation',
                                    valid_pct=0.2,
                                    shuffle=True)

# กำหนด backbone ที่เราต้องการใช้งานให้กับ cnn_learner เช่น resnet18
learner = cnn_learner(dls, 
                    resnet18, 
                    metrics=[accuracy, error_rate])

learner.fine_tune(4)

# **Augmentations**

In [ ]:
import torchvision.transforms as T

train_transform = T.Compose([
    T.Resize((256, 256)),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(),
    T.RandomResizedCrop((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),)
])

In [ ]:
train_data = AnimalDataset(train_filenames, train_labels, transform=train_transform)
val_data = AnimalDataset(val_filenames, val_labels, transform=val_transform)

# **มาทดลองการจำแนกรูปภาพกัน**

Taken from [AI builders' repository](https://github.com/ai-builders/curriculum/blob/main/notebooks/04v_classification_pytorch.ipynb)

**Authored by**: Titipat

In [ ]:
## upload your kaggle.json file to colab workspace first
## before running this cell

!mkdir /root/.kaggle
!cp kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# SOURCE: https://www.kaggle.com/competitions/dog-breed-identification/data

!kaggle competitions download -c dog-breed-identification
!unzip dog-breed-identification.zip -d data

In [ ]:
# !pip install pytorch_lightning

In [ ]:
import os
import os.path as op
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# get all image paths
img_df = pd.DataFrame(glob("data/train/*.jpg"), columns=["path"])
img_df["id"] = img_df.path.map(lambda x: op.basename(x).replace(".jpg", ""))

# read label data
label_df = pd.read_csv("data/labels.csv")
train_df = img_df.merge(label_df, on="id")

In [ ]:
train_df.head()

,path,id,breed
0,data/train/ec3fd4eea9a6a2c88908c33737442e4a.jpg,ec3fd4eea9a6a2c88908c33737442e4a,kerry_blue_terrier
1,data/train/700ef49936ff04f8490ceff02b01127f.jpg,700ef49936ff04f8490ceff02b01127f,leonberg
2,data/train/c9ea2b424b0074a33ec7a879b1cb25ca.jpg,c9ea2b424b0074a33ec7a879b1cb25ca,rottweiler
3,data/train/9108f7ab07ed5d12dd618f604867ed75.jpg,9108f7ab07ed5d12dd618f604867ed75,old_english_sheepdog
4,data/train/0518691772e78ac6805bf006993665a4.jpg,0518691772e78ac6805bf006993665a4,rhodesian_ridgeback


In [ ]:
train_df, validation_df = train_test_split(train_df, test_size=0.2, random_state=3)

In [ ]:
print("Length of training set = {}, validation set = {}".format(len(train_df), len(validation_df)))

Length of training set = 8177, validation set = 2045


In [ ]:
root_dir = "data/dogdata/"
for df, f in zip([train_df, validation_df], ["train", "validation"]):
    for _, r in df.iterrows():
        # create subfolder if it doesn't exist
        d = op.join(root_dir, f, r.breed)
        if not op.exists(d):
            os.makedirs(d)
        shutil.copy(r.path, op.join(root_dir, f, r.breed, f"{r.id}.jpg"))

### **Image classification ด้วย Pytorch Lightning**

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
train_transform = T.Compose([
    T.Resize((256, 256)),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(),
    T.RandomResizedCrop((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),)
])

In [ ]:
train_data = datasets.ImageFolder("data/dogdata/train/", transform=train_transform)
val_data = datasets.ImageFolder("data/dogdata/validation/", transform=val_transform)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
classes = train_data.classes
n_classes = len(classes)

In [ ]:
class DogResNet(pl.LightningModule):
    def __init__(self, n_classes=120):
        super(DogResNet, self).__init__()
        
        # จำนวนของพันธุ์น้องหมา (120)
        self.n_classes = n_classes

        # ใช้สถาปัตยกรรม resnet34; เปลี่ยน layer สุดท้าย
        self.backbone = models.resnet34(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False
        # เปลี่ยน fc layer เป็น output ขนาด 120
        self.backbone.fc = torch.nn.Linear(self.backbone.fc.in_features, n_classes)
        
        self.entropy_loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy()

    def forward(self, x):
        preds = self.backbone(x)
        return preds

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.backbone(x)
        loss = self.entropy_loss(logits, y)
        y_pred = torch.argmax(logits, dim=1)
        
        # log metrics ที่สำคัญไว้เพื่อการวิเคราะห์ในภายหลัง
        self.log("train_loss", loss)
        self.log("train_acc", self.accuracy(y_pred, y))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.backbone(x)
        loss = self.entropy_loss(logits, y)
        y_pred = torch.argmax(logits, dim=1)
        
        # log metrics ที่สำคัญไว้เพื่อการวิเคราะห์ในภายหลัง
        self.log("val_loss", loss)
        self.log("val_acc", self.accuracy(y_pred, y))
        return loss
        
    def configure_optimizers(self):
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        return {
            "optimizer": self.optimizer,
            "monitor": "val_loss",
        }

In [ ]:
model = DogResNet(n_classes=n_classes)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
# callback เพื่อให้ Trainer เซฟโมเดลไว้ในไฟล์ checkpoint เมื่อผลลัพธ์ val_loss ลดต่ำกว่าที่เคย
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints/dogbreed/",
    filename="resnet18--{epoch:02d}-{val_acc:.2f}-{val_loss:.2f}",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

In [ ]:
trainer = pl.Trainer(max_epochs=10, gpus=1, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params
--------------------------------------------------
0 | backbone     | ResNet           | 21.3 M
1 | entropy_loss | CrossEntropyLoss | 0     
2 | acc

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 256: 'val_loss' reached 1.47828 (best 1.47828), saving model to '/content/checkpoints/dogbreed/resnet18--epoch=00-val_acc=0.64-val_loss=1.48.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 2048: 'val_loss' reached 0.70046 (best 0.70046), saving model to '/content/checkpoints/dogbreed/resnet18--epoch=07-val_acc=0.78-val_loss=0.70.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 2304: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 2560: 'val_loss' reached 0.70019 (best 0.70019), saving model to '/content/checkpoints/dogbreed/resnet18--epoch=09-val_acc=0.78-val_loss=0.70.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
